# Angle recalculations

The angular deviation between two vectors will be reconstructed using two methods:
   - The acute angular method: Calculated by applying cosine.
   - The obtuse angular method: Calculated by applying atan2.
   
 

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
import math
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load corrected integrated Pointing task

Data comes as a result of script `02_ParticipantsDataReconstruction`

In [2]:
CoordinatesIntegrated = pd.read_csv('/Volumes/TwoTeras/0_Experiment_1/Pointing_Task/Results/PointingTask_Integrated_exp1.csv')
CoordinatesIntegrated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     10080 non-null  int64  
 1   SubjectID                      10080 non-null  int64  
 2   ParticipantPosition_x          10080 non-null  float64
 3   ParticipantPosition_z          10080 non-null  float64
 4   PointerPosition_x              10080 non-null  float64
 5   PointerPosition_z              10080 non-null  float64
 6   PointerDirection_x             10080 non-null  float64
 7   PointerDirection_z             10080 non-null  float64
 8   BuildingCenterWorld_z          10080 non-null  float64
 9   BuildingCenterWorld_x          10080 non-null  float64
 10  DistanceToParticipant          10080 non-null  float64
 11  PointingTaskStartingLocations  10080 non-null  int64  
 12  AgentID                        10080 non-null 

In [3]:
len(CoordinatesIntegrated.SubjectID.unique())

30

## Creation of vectors

In [4]:
CoordinatesIntegrated['vector_PartPos'] = list(zip(CoordinatesIntegrated.ParticipantPosition_x, CoordinatesIntegrated.ParticipantPosition_z))
CoordinatesIntegrated['vector_BuildingPos'] = list(zip(CoordinatesIntegrated['BuildingCenterWorld_x'], CoordinatesIntegrated['BuildingCenterWorld_z']))
CoordinatesIntegrated['vector_PointerPos'] =list(zip(CoordinatesIntegrated.PointerDirection_x, CoordinatesIntegrated.PointerDirection_z))

In [5]:
PartList = list(CoordinatesIntegrated['vector_PartPos'])
BuiltList = list(CoordinatesIntegrated['vector_BuildingPos'])
PointList = list(CoordinatesIntegrated['vector_PointerPos'])

In [6]:
### Calculating direction vector from Participant Position (a) to Target Building Position (b): b- **  component form is terminal minus initial
CoordinatesIntegrated['Vector_Participant-Building'] = [tuple(x-y for x, y in zip(tup1, tup2)) for tup1, tup2 in zip(BuiltList, PartList)]
### Calculating direction vector from Participant Position (a) to pointer (b): a-b
CoordinatesIntegrated['Vector_Participant-Pointer'] = [tuple(x-y for x, y in zip(tup1, tup2)) for tup1, tup2 in zip(PointList, PartList)] # addition
CoordinatesIntegrated.iloc[:,-15:-8].head(5)

,Trials,TrialNumber,StartPointID.1,ID_for_StartingPosition,ImageName,TimeDelta,RT
0,90,1,1,1,46_Sa,0 days 00:00:25.825714432,25.825715
1,91,78,7,6,46_SaNo,0 days 00:00:04.799789312,4.799789
2,90,88,8,4,46_Sa,0 days 00:00:04.638590208,4.638590
3,91,95,8,11,46_SaNo,0 days 00:00:02.361455104,2.361455
4,91,131,11,11,46_SaNo,0 days 00:00:05.975303680,5.975304


In [7]:
CoordinatesIntegrated.groupby(['BuildingCenterWorld_z','BuildingCenterWorld_x',  'ImageName']).count()

Unnamed: 0  SubjectID  \
BuildingCenterWorld_z BuildingCenterWorld_x ImageName                          
-255.873932           -3.336735             23_CmA            106        106   
                                            23_CmANo           83         83   
-205.836075           -42.972923            48_Sa              94         94   
                                            48_SaNo           100        100   
-204.096573            96.337585            40_Sa              85         85   
...                                                           ...        ...   
 226.580154           -72.436798            18_CmANo           76         76   
 234.419739            197.370117           03_CmA             74         74   
                                            03_CmANo           93         93   
 288.053894           -234.506821           56_Sa              84         84   
                                            56_SaNo            97         97   

                                                       ParticipantPosition_x  \
BuildingCenterWorld_z BuildingCenterWorld_x ImageName                          
-255.873932           -3.336735             23_CmA                       106   
                                            23_CmANo                      83   
-205.836075           -42.972923            48_Sa                         94   
                                            48_SaNo                      100   
-204.096573            96.337585            40_Sa                         85   
...                                                                      ...   
 226.580154           -72.436798            18_CmANo                      76   
 234.419739            197.370117           03_CmA                        74   
                                            03_CmANo                      93   
 288.053894           -234.506821           56_Sa                         84   
                                            56_SaNo                       97   

                                                       ParticipantPosition_z  \
BuildingCenterWorld_z BuildingCenterWorld_x ImageName                          
-255.873932           -3.336735             23_CmA                       106   
                                            23_CmANo                      83   
-205.836075           -42.972923            48_Sa                         94   
                                            48_SaNo                      100   
-204.096573            96.337585            40_Sa                         85   
...                                                                      ...   
 226.580154           -72.436798            18_CmANo                      76   
 234.419739            197.370117           03_CmA                        74   
                                            03_CmANo                      93   
 288.053894           -234.506821           56_Sa                         84   
                                            56_SaNo                       97   

                                                       PointerPosition_x  \
BuildingCenterWorld_z BuildingCenterWorld_x ImageName                      
-255.873932           -3.336735             23_CmA                   106   
                                            23_CmANo                  83   
-205.836075           -42.972923            48_Sa                     94   
                                            48_SaNo                  100   
-204.096573            96.337585            40_Sa                     85   
...                                                                  ...   
 226.580154           -72.436798            18_CmANo                  76   
 234.419739            197.370117           03_CmA                    74   
                                            03_CmANo                  93   
 288.053894           -234.506821           56_Sa                     84   
                    

## Angle functions acute method

In [8]:
def DotTuple(v1,v2):
    """Returns the dot product of two vectors of two elements each
Parameters
-------------
    v1: tuple of two elements
    v2: tuple of two elements
returns
----------
    list of dot product of tuples
"""
    dotList = []
    dot = [sum(x*y for x, y in zip(tup1, tup2))for tup1, tup2 in zip(v1, v2)]
    dotList.append(dot)
    flatlist = [item for sublist in dotList for item in sublist]
    return flatlist

In [9]:
def Magnitud(v1):
    """Returns the magnitute of a vector
Parameters
-------------
    v1: tuple of two elements
returns
----------
    list with magnitude of vectors
"""
    v1 = [x ** 2 + y ** 2 for x,y in v1]
    squared = [math.sqrt(value) for value in v1]
    return(squared)

In [10]:
CoordinatesIntegrated['building-Participant_x'] = CoordinatesIntegrated['BuildingCenterWorld_x'] -  CoordinatesIntegrated['ParticipantPosition_x']
CoordinatesIntegrated['building-Participant_z'] = CoordinatesIntegrated['BuildingCenterWorld_z'] - CoordinatesIntegrated['ParticipantPosition_z']

In [11]:
# Creating the individual elements of the arc cos formula and inserting them on the dataframe
CoordinatesIntegrated['Dot_Pointer-Building-Participant'] = DotTuple(CoordinatesIntegrated['Vector_Participant-Pointer'],CoordinatesIntegrated['Vector_Participant-Building'])
CoordinatesIntegrated['Magnitud_Participant-Pointer'] = Magnitud(CoordinatesIntegrated['Vector_Participant-Pointer'])
CoordinatesIntegrated['Magnitud_Participant-Building'] = Magnitud(CoordinatesIntegrated['Vector_Participant-Building'])

In [12]:
# Calculating degrees using the arc cos formula and inserting them on the dataframe
my_angle = CoordinatesIntegrated['Dot_Pointer-Building-Participant']/ (CoordinatesIntegrated['Magnitud_Participant-Pointer'] * CoordinatesIntegrated['Magnitud_Participant-Building'])
CoordinatesIntegrated['radians'] =  [math.acos(value) for value in my_angle]
CoordinatesIntegrated['degrees'] = [np.rad2deg(value) for value in CoordinatesIntegrated['radians']]
CoordinatesIntegrated['degrees'].describe()

count    10080.000000
mean        42.885166
std         38.179486
min          0.006036
25%         14.470790
50%         31.236033
75%         60.094427
max        179.736428
Name: degrees, dtype: float64

## Angle functions obtuse method

In [13]:
def signedAngle(v1, v2):
    """" Returns the subtraction of arc tangent of vector 2 minus vector 1
Parameters
-------------
v1: tuple of values that contains the coordinates of your reference initial vector,
    in our case the perfect angle between position and target (remember to order them y,x)
v2: tuple of values that contains the coordinates of your vector of contrast,
    in our case the real reply from the participants (remember to order them y,x)
returns
----------

"""
    angle = math.atan2(v2[1], v2[0]) - math.atan2(v1[1], v1[0])
    return(np.rad2deg(angle))

In [14]:
signedAngles = []

for count, x in enumerate(CoordinatesIntegrated['PointerDirection_z']):
    signedAngles = signedAngles + [signedAngle([ CoordinatesIntegrated['PointerDirection_z'][count], CoordinatesIntegrated['PointerDirection_x'][count]], [CoordinatesIntegrated['building-Participant_z'][count]
       , CoordinatesIntegrated['building-Participant_x'][count]])]

# Account for angle deviations > 180°
for count, angle in enumerate(signedAngles):
    if angle < 0:
        signedAngles[count] = angle + 360
CoordinatesIntegrated.insert(11, "SignedAngle360", signedAngles, True)

for count, angle in enumerate(signedAngles):
    if angle > 180:
        signedAngles[count] = angle - 360
    elif angle < -180:
        signedAngles[count] = angle + 360
CoordinatesIntegrated.insert(11, "SignedAngle-+180", signedAngles, True)
CoordinatesIntegrated['SignedAngle-+180'].describe()

count    10080.000000
mean        -5.736533
std         66.032976
min       -179.881808
25%        -40.724704
50%         -5.444933
75%         25.867365
max        179.983386
Name: SignedAngle-+180, dtype: float64

In [15]:
PointerZ = CoordinatesIntegrated.loc[:,'PointerDirection_z'].values
PointerX = CoordinatesIntegrated.loc[:,'PointerDirection_x'].values
BuildingParticipantZ = CoordinatesIntegrated.loc[:,'building-Participant_z'].values
BuldingParticipantX= CoordinatesIntegrated.loc[:,'building-Participant_x'].values

In [16]:
CoordinatesIntegrated['atanPointer'] = [math.atan2(z, x) for z, x in zip(PointerZ , PointerX)]
CoordinatesIntegrated['atanBuild-Part'] = [math.atan2(z, x) for z, x in zip(BuildingParticipantZ , BuldingParticipantX)]
CoordinatesIntegrated['atanSubs'] = CoordinatesIntegrated['atanPointer'] - CoordinatesIntegrated['atanBuild-Part']
CoordinatesIntegrated['rad2degree']= [np.rad2deg(value) for value in CoordinatesIntegrated['atanSubs']]
signedAngles = CoordinatesIntegrated['rad2degree'].values

In [17]:
for count, angle in enumerate(signedAngles):
    if angle > 180:
        signedAngles[count] = angle - 360
    elif angle < -180:
        signedAngles[count] = angle + 360
CoordinatesIntegrated.insert(11, "SignedAngle-+1802", signedAngles, True)

In [18]:
CoordinatesIntegrated['AbsolutError'] = abs(CoordinatesIntegrated['rad2degree'])
CoordinatesIntegrated[['AbsolutError']].describe()

,AbsolutError
count,10080.000000
mean,48.772063
std,44.887741
min,0.009861
25%,12.788525
50%,34.385003
75%,72.651919
max,179.983386


## Identifying extreme values and flagging first trials

# Flagging the absolute values that transcend 1.5 times and 3 times the inter-quartile range

In [19]:
def find_iqr(x):
    """"
    Returns inter-quartile range of a numeric (i.e, integer or float)
Parameters
-------------
    x: Numeric (i.e, integer or float) variable one wants to calculate interquartile range from
returns
----------
    float that contains interquartile range
    """
    return np.subtract(*np.percentile(x, [75, 25]))


In [20]:
# Create and empty dataframe that will countain interquartile range
inter_quartile_range = pd.DataFrame(columns=['SubjectID', 'IQR'])

#Loop through all subjects and calculate personal IQR
for i, subject in enumerate(CoordinatesIntegrated.SubjectID.unique()):
    inter_quartile_range.at[i, 'SubjectID'] = subject
    inter_quartile_range.at[i, 'IQR'] = find_iqr(CoordinatesIntegrated[CoordinatesIntegrated.SubjectID == subject]['AbsolutError'])

inter_quartile_range['IQRlimit'] = inter_quartile_range['IQR'] * 1.5
inter_quartile_range['IQRlimit2'] = inter_quartile_range['IQR'] * 3

inter_quartile_range.sort_values('IQR')

,SubjectID,IQR,IQRlimit,IQRlimit2
13,4917,15.933994,23.90099,47.801981
10,4176,22.212112,33.318168,66.636336
20,7412,22.267043,33.400564,66.801128
1,479,25.855071,38.782607,77.565214
3,2258,31.996184,47.994275,95.988551
28,9586,41.605113,62.40767,124.81534
17,6642,41.924538,62.886807,125.773614
6,3246,43.714881,65.572322,131.144644
4,2361,45.523749,68.285624,136.571248
9,3976,48.755161,73.132742,146.265483


In [21]:
# Merged finished data base with inter-quartile range base
interQ = pd.merge(inter_quartile_range,CoordinatesIntegrated, on='SubjectID')
interQ.head(50)

,SubjectID,IQR,IQRlimit,IQRlimit2,Unnamed: 0,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,PointerDirection_x,...,Dot_Pointer-Building-Participant,Magnitud_Participant-Pointer,Magnitud_Participant-Building,radians,degrees,atanPointer,atanBuild-Part,atanSubs,rad2degree,AbsolutError
0,365,63.921884,95.882826,191.765652,0,-114.329147,-139.997375,-114.236984,-139.696228,-0.055459,...,32021.797071,181.482199,197.743621,0.468390,26.836743,1.626782,1.358059,0.268723,15.396716,15.396716
1,365,63.921884,95.882826,191.765652,2,-65.669235,-159.945099,-65.474716,-159.710205,0.072550,...,33863.023381,173.847081,213.345438,0.420196,24.075485,1.497808,1.603186,-0.105378,-6.037733,6.037733
2,365,63.921884,95.882826,191.765652,4,38.849380,53.547638,38.471634,53.516281,-0.993265,...,4453.481807,66.834707,111.427967,0.929789,53.272987,-3.028520,-3.139266,0.110746,6.345290,6.345290
3,365,63.921884,95.882826,191.765652,6,38.880081,53.564148,38.700195,53.581047,-0.995648,...,4459.269897,66.829809,111.458707,0.928970,53.226082,-3.075803,-3.139119,0.063316,3.627752,3.627752
4,365,63.921884,95.882826,191.765652,8,110.754181,196.522675,110.428146,196.550751,-0.941145,...,48672.268414,226.326423,232.651757,0.391448,22.428323,-2.811994,-2.478370,-0.333624,-19.115237,19.115237
5,365,63.921884,95.882826,191.765652,10,-228.862320,189.713608,-228.800003,189.430649,0.052264,...,61793.390879,297.946867,207.452469,0.023052,1.320792,-1.518411,-0.717657,-0.800754,-45.879831,45.879831
6,365,63.921884,95.882826,191.765652,12,257.815582,-33.150925,257.569214,-33.181824,-0.990357,...,88383.092609,260.934694,341.514086,0.128067,7.337709,3.029311,2.885703,0.143608,8.228125,8.228125
7,365,63.921884,95.882826,191.765652,14,138.625366,-128.793640,138.342194,-128.611328,-0.886482,...,52999.822407,190.182931,278.856352,0.035755,2.048609,2.664697,2.430106,0.234591,13.441060,13.441060
8,365,63.921884,95.882826,191.765652,16,366.767181,108.779312,366.540100,108.767654,-0.957100,...,167609.469643,383.554183,442.835947,0.162663,9.319878,-2.862593,-3.015954,0.153362,8.786976,8.786976
9,365,63.921884,95.882826,191.765652,18,366.760132,108.726776,366.563049,108.749207,-0.859302,...,167563.226130,383.495456,442.822373,0.163223,9.351997,-2.637894,-3.016070,0.378176,21.667869,21.667869


In [22]:
## Add flag that's true if the puntuaction should stay false if it should go
interQ['Flag'] =  [limit > absolut  for absolut,limit in zip(interQ['AbsolutError'],interQ['IQRlimit'])]
interQ['Flag2'] =  [limit > absolut  for absolut,limit in zip(interQ['AbsolutError'],interQ['IQRlimit2'])]
interQ.head(50)

,SubjectID,IQR,IQRlimit,IQRlimit2,Unnamed: 0,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,PointerDirection_x,...,Magnitud_Participant-Building,radians,degrees,atanPointer,atanBuild-Part,atanSubs,rad2degree,AbsolutError,Flag,Flag2
0,365,63.921884,95.882826,191.765652,0,-114.329147,-139.997375,-114.236984,-139.696228,-0.055459,...,197.743621,0.468390,26.836743,1.626782,1.358059,0.268723,15.396716,15.396716,True,True
1,365,63.921884,95.882826,191.765652,2,-65.669235,-159.945099,-65.474716,-159.710205,0.072550,...,213.345438,0.420196,24.075485,1.497808,1.603186,-0.105378,-6.037733,6.037733,True,True
2,365,63.921884,95.882826,191.765652,4,38.849380,53.547638,38.471634,53.516281,-0.993265,...,111.427967,0.929789,53.272987,-3.028520,-3.139266,0.110746,6.345290,6.345290,True,True
3,365,63.921884,95.882826,191.765652,6,38.880081,53.564148,38.700195,53.581047,-0.995648,...,111.458707,0.928970,53.226082,-3.075803,-3.139119,0.063316,3.627752,3.627752,True,True
4,365,63.921884,95.882826,191.765652,8,110.754181,196.522675,110.428146,196.550751,-0.941145,...,232.651757,0.391448,22.428323,-2.811994,-2.478370,-0.333624,-19.115237,19.115237,True,True
5,365,63.921884,95.882826,191.765652,10,-228.862320,189.713608,-228.800003,189.430649,0.052264,...,207.452469,0.023052,1.320792,-1.518411,-0.717657,-0.800754,-45.879831,45.879831,True,True
6,365,63.921884,95.882826,191.765652,12,257.815582,-33.150925,257.569214,-33.181824,-0.990357,...,341.514086,0.128067,7.337709,3.029311,2.885703,0.143608,8.228125,8.228125,True,True
7,365,63.921884,95.882826,191.765652,14,138.625366,-128.793640,138.342194,-128.611328,-0.886482,...,278.856352,0.035755,2.048609,2.664697,2.430106,0.234591,13.441060,13.441060,True,True
8,365,63.921884,95.882826,191.765652,16,366.767181,108.779312,366.540100,108.767654,-0.957100,...,442.835947,0.162663,9.319878,-2.862593,-3.015954,0.153362,8.786976,8.786976,True,True
9,365,63.921884,95.882826,191.765652,18,366.760132,108.726776,366.563049,108.749207,-0.859302,...,442.822373,0.163223,9.351997,-2.637894,-3.016070,0.378176,21.667869,21.667869,True,True


In [23]:
interQ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 58 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   SubjectID                         10080 non-null  object 
 1   IQR                               10080 non-null  object 
 2   IQRlimit                          10080 non-null  object 
 3   IQRlimit2                         10080 non-null  object 
 4   Unnamed: 0                        10080 non-null  int64  
 5   ParticipantPosition_x             10080 non-null  float64
 6   ParticipantPosition_z             10080 non-null  float64
 7   PointerPosition_x                 10080 non-null  float64
 8   PointerPosition_z                 10080 non-null  float64
 9   PointerDirection_x                10080 non-null  float64
 10  PointerDirection_z                10080 non-null  float64
 11  BuildingCenterWorld_z             10080 non-null  float64
 12  Buil

In [24]:
interQ.Flag.value_counts()

Flag
True     8060
False    2020
Name: count, dtype: int64

In [25]:
interQ.Flag2.value_counts()

Flag2
True     9814
False     266
Name: count, dtype: int64

In [26]:
CoordinatesIntegrated.columns

Index(['Unnamed: 0', 'SubjectID', 'ParticipantPosition_x',
       'ParticipantPosition_z', 'PointerPosition_x', 'PointerPosition_z',
       'PointerDirection_x', 'PointerDirection_z', 'BuildingCenterWorld_z',
       'BuildingCenterWorld_x', 'DistanceToParticipant', 'SignedAngle-+1802',
       'SignedAngle-+180', 'SignedAngle360', 'PointingTaskStartingLocations',
       'AgentID', 'Name', 'StartPointID', 'PointingTargetCondition', 'Context',
       'Agent', 'avatarID', 'AvatarPresence', 'AvatarPresenceCategory',
       'meaningful', 'meaningfulBuilding', 'Trials', 'TrialNumber',
       'StartPointID.1', 'ID_for_StartingPosition', 'ImageName', 'TimeDelta',
       'RT', 'TimeOut', 'TimeStampBegin', 'TimeStampEnd', 'vector_PartPos',
       'vector_BuildingPos', 'vector_PointerPos',
       'Vector_Participant-Building', 'Vector_Participant-Pointer',
       'building-Participant_x', 'building-Participant_z',
       'Dot_Pointer-Building-Participant', 'Magnitud_Participant-Pointer',
       'M

## First pointing trial

In our design the participants have to point to targets from 28 starting locations, they switch positions every 12 trials. It would be sensible to assume that the first trial in each location can count more variance of error since they are starting to familiarize themselves with the environment.

In [27]:
# Flag the first trial of every starting position.
# This has the limitation that it wrongly flags for the last trial of first pointing position
interQ['Removefirst'] = [not (value % 12 == 0) for value in interQ['TrialNumber']]
# The not operator it's used in this case so that all trials that stay are true and the ones to be eliminated are false
interQ[['TrialNumber', 'Removefirst']].head(13)

,TrialNumber,Removefirst
0,1,True
1,78,True
2,88,True
3,95,True
4,131,True
5,137,True
6,146,True
7,180,False
8,289,True
9,292,True


In [28]:
# Here we correct for the mistake the previous method created (i.e., deflag the 12th trial)
interQ.loc[interQ['TrialNumber'] == 12, 'Removefirst'] = True
# And we flag all first trials
interQ.loc[interQ['TrialNumber'] == 1, 'Removefirst'] = False

In [29]:
# Proof of concept: Here we retain only the flagged trials and check if the correct Trial Ids have been identified
TeoryFirst = interQ[interQ['Removefirst'] == False].copy()
check = pd.DataFrame(TeoryFirst['TrialNumber'].unique())
check.describe()

,0
count,28.000000
mean,173.607143
std,99.399806
min,1.000000
25%,93.000000
50%,174.000000
75%,255.000000
max,336.000000


In [30]:
# Added cell to set Working Directory to your location
os.chdir('/Volumes/TwoTeras/0_Experiment_1/Pointing_Task/Results/')
os.getcwd()
interQ.to_csv('PointingTask_IQR_Final_exp1.csv')

In [31]:
interQ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 59 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   SubjectID                         10080 non-null  object 
 1   IQR                               10080 non-null  object 
 2   IQRlimit                          10080 non-null  object 
 3   IQRlimit2                         10080 non-null  object 
 4   Unnamed: 0                        10080 non-null  int64  
 5   ParticipantPosition_x             10080 non-null  float64
 6   ParticipantPosition_z             10080 non-null  float64
 7   PointerPosition_x                 10080 non-null  float64
 8   PointerPosition_z                 10080 non-null  float64
 9   PointerDirection_x                10080 non-null  float64
 10  PointerDirection_z                10080 non-null  float64
 11  BuildingCenterWorld_z             10080 non-null  float64
 12  Buil

In [32]:
#Elimininating trails were the participant ran out of time
Clean = interQ[interQ['TimeOut']== False]

In [33]:
PointingSum = pd.DataFrame(Clean[['SubjectID', 'AbsolutError', 'SignedAngle-+180', 'IQR', 'RT',  'DistanceToParticipant', 'PointingTaskStartingLocations',  'TrialNumber', 'StartPointID', 'ID_for_StartingPosition','avatarID', 'AvatarPresenceCategory', 'Context', 'meaningfulBuilding',  'ImageName']].values)

In [34]:
PointingSum.columns = ['ID', 'AbsolutError', 'SignedAngle-+180', 'IQR', 'RT',  'DistanceToParticipant', 'PointingTaskStartingLocations',  'TrialNumber', 'StartPointID', 'ID_for_StartingPosition','avatarID', 'AvatarPresenceCategory', 'Context', 'meaningfulBuilding',  'ImageName']
PointingSum.head()

,ID,AbsolutError,SignedAngle-+180,IQR,RT,DistanceToParticipant,PointingTaskStartingLocations,TrialNumber,StartPointID,ID_for_StartingPosition,avatarID,AvatarPresenceCategory,Context,meaningfulBuilding,ImageName
0,365,15.396716,15.396716,63.921884,25.825715,197.768723,21,1,1,1,46,Present,False,Not meaningful,46_Sa
1,365,6.037733,-6.037733,63.921884,4.799789,213.475525,16,78,7,6,46,Omitted,False,Not meaningful,46_SaNo
2,365,6.34529,6.34529,63.921884,4.63859,111.623512,13,88,8,4,46,Present,False,Not meaningful,46_Sa
3,365,3.627752,3.627752,63.921884,2.361455,111.623512,13,95,8,11,46,Omitted,False,Not meaningful,46_SaNo
4,365,19.115237,-19.115237,63.921884,5.975304,232.765823,8,131,11,11,46,Omitted,False,Not meaningful,46_SaNo


In [35]:
PointingSum.to_csv('PointingSummary_1_exp1.csv')
Clean.to_csv('FULL_Pointing_exp1.csv')

In [36]:
PointingSum[['ImageName', 'avatarID']]

,ImageName,avatarID
0,46_Sa,46
1,46_SaNo,46
2,46_Sa,46
3,46_SaNo,46
4,46_SaNo,46
...,...,...
10043,53_SaNo,53
10044,53_Sa,53
10045,53_SaNo,53
10046,53_Sa,53


In [37]:
PointingSum.ID.unique()

array([365, 479, 1754, 2258, 2361, 2693, 3246, 3310, 3572, 3976, 4176,
       4597, 4796, 4917, 5238, 5531, 5741, 6642, 7093, 7264, 7412, 7842,
       8007, 8469, 8673, 8695, 9472, 9502, 9586, 9601], dtype=object)